In [1]:
import pandas as pd
import plotly.express as px
import os
from dotenv import load_dotenv

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [2]:


load_dotenv()

mongodb_URI = os.getenv("mongodb_URI")

In [3]:
uri = mongodb_URI


In [4]:
client = MongoClient(uri, server_api=ServerApi('1'))

In [5]:
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Server at localhost:27017 reports wire version 6, but this version of PyMongo requires at least 7 (MongoDB 4.0).


In [6]:
db = client['test']
purchase_collection = db['purchasings']

In [7]:
cursor = purchase_collection.find({}, {
    "ticker": 1,
    "purchaseValue": 1,
    "quantity": 1,
    "purchaseDate": 1,
    "_id": 0
})

In [8]:
data = list(cursor)

ConfigurationError: Server at localhost:27017 reports wire version 6, but this version of PyMongo requires at least 7 (MongoDB 4.0).

In [ ]:
df = pd.DataFrame(data)
df

,ticker,purchaseValue,quantity,purchaseDate
0,BBAS3,21.59,2,2025-06-13
1,BBAS3,21.59,1,2025-06-17
2,BBAS3,21.59,4,2025-06-18


In [ ]:
# Calcular o valor total investido por linha
df['totalInvestido'] = df['purchaseValue'] * df['quantity']

# Agrupar por data e somar os valores investidos
df_grouped = df.groupby(['purchaseDate', 'ticker']).agg({'totalInvestido': 'sum'}).reset_index()

fig1 = px.line(
    df_grouped,
    x='purchaseDate',
    y='totalInvestido',
    color='ticker',
    title='Investimentos por dia por ativo',
    markers=True,
    labels={'purchaseDate': 'Data da compra', 'totalInvestido': 'Total investido (R$)', 'ticker': 'Ativo'}
)

fig1.show()

In [ ]:
df_grouped

,purchaseDate,ticker,totalInvestido
0,2025-06-13,BBAS3,43.18
1,2025-06-17,BBAS3,21.59
2,2025-06-18,BBAS3,86.36


In [ ]:
df_grouped = df.groupby(['purchaseDate', 'ticker']).agg({'quantity': 'sum'}).reset_index()

fig2 = px.line(
    df_grouped,
    x='purchaseDate',
    y='quantity',
    color='ticker',
    title='Quantidade total comprada por dia',
    markers=True,
    labels={'purchaseDate': 'Data', 'quantity': 'Quantidade', 'ticker': 'Ativo'}
)
fig2.show()